In [ ]:
import pandas as pd
from os import path
from selenium import webdriver
from trafilatura import extract


def query(string):
    for column in db.columns:
        contents = [i for i in db[column].dropna()
                    if str(string).lower() in i.lower()]
        if contents:
            return contents
        else:
            pass
    print('No match')

In [ ]:
if path.isfile('db.h5'):
    db = pd.read_hdf('db.h5')
else:
    db = pd.DataFrame({},columns=['Article Text','Author','Headline','Article URL','Source Name'])

In [ ]:
driver = webdriver.Chrome()

driver.get('https://www.bloomberg.com/technology')

In [ ]:
elements = driver.find_elements_by_xpath('//a[contains(@href, "%s")]' % '/articles/')

for article in elements:
    entry = len(db)
    headline = article.text.split('\n')[0]
    if headline:
        db.loc[entry, 'Headline'] = headline
        db.loc[entry, 'Article URL'] = article.get_attribute('href').split('?')[0]
        db.loc[entry, 'Source Name'] = 'Bloomberg Technology'

bloomberglinks = db[(db['Source Name'] == 'Bloomberg Technology')
                    & pd.isna(db['Article Text'])].index

for link in bloomberglinks:
    try:
        driver.get(db['Article URL'][link])
        keys = driver.find_element_by_xpath('//*[address]')

        for values in keys.text.split('\n'):
            if values[0:2] == 'By':
                db.loc[link, 'Author'] = values[3:]

        sleep(1)

        db.loc[link, 'Article Text'] = extract(driver.page_source) 
    except:
        pass

driver.Quit()

In [ ]:
# sometimes there are duplicates articles in this webpage
db = db.drop_duplicates().reset_index().drop('index',axis=1)

In [ ]:
driver = webdriver.Chrome()
driver.get('https://elcomercio.pe/economia/?ref=ecr')

In [ ]:
elements = driver.find_elements_by_xpath('//h2[a]')
authors = driver.find_elements_by_xpath('//a[contains(@href, "%s")]' % '/autor')
links = driver.find_elements_by_xpath('//h2//*[@href]')

for i, headline in enumerate(elements):
    entry = len(db)
    db.loc[entry, 'Headline'] = headline.text
    db.loc[entry, 'Author'] = authors[i].text
    db.loc[entry, 'Article URL'] = links[i].get_attribute('href')
    db.loc[entry, 'Source Name'] = 'El Comercio'

elcomerciolinks = db[(db['Source Name'] == 'El Comercio')
                     & pd.isna(db['Article Text'])].index

for link in elcomerciolinks:
    try:
        driver.get(db['Article URL'][link])
        
        sleep(1)
        
        db.loc[link, 'Article Text'] = extract(driver.page_source)
    except:
        pass

driver.Quit()

In [ ]:
db

In [ ]:
len(query('Alphabet'))

In [ ]:
query('Alphabet')[0]#.split('\n')

In [ ]:
db.to_hdf('db.h5', key='articles')